In [1]:
import ast
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.metrics import jaccard_score, hamming_loss
from collections import Counter


from transformers import (
    DistilBertModel, 
    DistilBertTokenizerFast,
    get_linear_schedule_with_warmup
)
from tqdm.auto import tqdm


/Users/malgosielska/movie-genre-keyword-prediction/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [3]:
data_file = "../movie_dataset_prepared.csv"
df = pd.read_csv(data_file)

In [4]:
df['keywords_list'] = df['keywords_list'].apply(ast.literal_eval)

print(type(df['keywords_list'][0]))
df.head()


<class 'list'>


,genres_list,keywords_list,overview
0,"[28, 12, 14, 878]","[9951, 209714]",in the 22nd century a paraplegic marine is dis...
1,"[12, 14, 28]",[179430],captain barbossa long believed to be dead has ...
2,"[28, 12, 80]","[818, 9663]",a cryptic message from bonds past sends him on...
3,"[28, 80, 18, 53]",[9715],following the death of district attorney harve...
4,"[28, 12, 878]","[818, 9951, 10685, 209714]",john carter is a warweary former military capt...


In [5]:
def clean_overview(x):
    if isinstance(x, str):
        return x
    if pd.isna(x):
        return ""
    return str(x)

df['overview'] = df['overview'].apply(clean_overview)


In [6]:
print(f"Przed usunieciem pustych keywords: {len(df)}")
df = df[df['keywords_list'].apply(lambda x: len(x) > 0)]
print(f"Po usunieciu pustych keywords: {len(df)}")

Przed usunieciem pustych keywords: 4683
Po usunieciu pustych keywords: 2720


In [7]:
texts = df['overview'].tolist() 
keywords_labels = df['keywords_list'].tolist()

texts, keywords_labels

(['in the 22nd century a paraplegic marine is dispatched to the moon pandora on a unique mission but becomes torn between following orders and protecting an alien civilization',
  'captain barbossa long believed to be dead has come back to life and is headed to the edge of the earth with will turner and elizabeth swann but nothing is quite as it seems',
  'a cryptic message from bonds past sends him on a trail to uncover a sinister organization while m battles political forces to keep the secret service alive bond peels back the layers of deceit to reveal the terrible truth behind spectre',
  'following the death of district attorney harvey dent batman assumes responsibility for dents crimes to protect the late attorneys reputation and is subsequently hunted by the gotham city police department eight years later batman encounters the mysterious selina kyle and the villainous bane a new terrorist leader who overwhelms gothams finest the dark knight resurfaces to protect a city that has 

In [8]:
all_keywords = [keyword for sublist in keywords_labels for keyword in sublist]
Counter(all_keywords)


Counter({187056: 310,
         10183: 310,
         179431: 307,
         818: 197,
         9826: 189,
         179430: 170,
         14819: 150,
         4565: 139,
         6075: 121,
         9748: 116,
         572: 111,
         6054: 106,
         4344: 104,
         5565: 102,
         13130: 99,
         209714: 96,
         9673: 95,
         9663: 94,
         9937: 92,
         242: 88,
         6149: 84,
         12670: 80,
         6270: 79,
         9951: 76,
         2483: 71,
         378: 70,
         9715: 69,
         212: 69,
         14964: 69,
         18035: 68,
         494: 65,
         3737: 64,
         5600: 61,
         1930: 57,
         1956: 57,
         13027: 57,
         9714: 56,
         236: 56,
         10714: 56,
         2343: 54,
         9713: 54,
         10685: 52,
         9717: 52,
         8508: 52,
         4379: 51,
         9672: 51,
         11612: 51,
         779: 51,
         417: 49})

In [18]:
mlb = MultiLabelBinarizer()
labels_encoded = mlb.fit_transform(keywords_labels)
n_classes = len(mlb.classes_)

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(
    texts, labels_encoded, test_size=0.15, random_state=42
)

In [9]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [10]:
class MovieDataset(Dataset):
    """Movie dataset for multi-label classification"""

    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(label)
        }

In [11]:
class BERTSingleTaskClassifier(nn.Module):
    def __init__(self, n_classes, dropout=0.3, bert_model='distilbert-base-uncased'):
        super(BERTSingleTaskClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained(bert_model)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        output = self.dropout(pooled_output)
        logits = self.classifier(output)
        return logits

In [12]:
def train_epoch(model, data_loader, criterion, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(data_loader, desc='Training')

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})

    return total_loss / len(data_loader)


In [13]:
@torch.no_grad()
def evaluate(model, loader, criterion, device, threshold=0.3):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0
    
    for batch in tqdm(loader, desc='Evaluating'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        preds = (torch.sigmoid(outputs) > threshold).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    metrics = {
        'loss': total_loss / len(loader),
        'f1_micro': f1_score(all_labels, all_preds, average='micro', zero_division=0),
        'f1_macro': f1_score(all_labels, all_preds, average='macro', zero_division=0),
        'precision': precision_score(all_labels, all_preds, average='micro', zero_division=0),
        'recall': recall_score(all_labels, all_preds, average='micro', zero_division=0),
        'hamming_accuracy': 1 - hamming_loss(all_labels, all_preds),
        'jaccard_samples': jaccard_score(all_labels, all_preds, average='samples')
    }
    return metrics

In [14]:
def run_single_experiment(
    config,
    X_trainval, y_trainval,
    X_test, y_test,
    tokenizer, n_classes, device
):
    BATCH_SIZE = config["batch_size"]
    EPOCHS = config["epochs"]
    LR = config["learning_rate"]
    DROPOUT = config["dropout"]
    THRESH = config["threshold"]

    # Split
    X_train, X_val, y_train, y_val = train_test_split(
        X_trainval, y_trainval, test_size=0.15, random_state=42
    )

    # Class weights
    counts = y_trainval.sum(axis=0)
    pos_weight = torch.tensor((len(y_trainval) - counts) / counts, dtype=torch.float)

    # Datasets
    train_loader = DataLoader(MovieDataset(X_train, y_train, tokenizer), batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(MovieDataset(X_val, y_val, tokenizer), batch_size=BATCH_SIZE)
    test_loader = DataLoader(MovieDataset(X_test, y_test, tokenizer), batch_size=BATCH_SIZE)

    # Model
    model = BERTSingleTaskClassifier(n_classes=n_classes, dropout=DROPOUT).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)
    optimizer = AdamW(model.parameters(), lr=LR)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=len(train_loader) * EPOCHS
    )

    best_f1 = 0.0
    best_state = None

    # Training loop
    for epoch in range(EPOCHS):
        train_epoch(model, train_loader, criterion, optimizer, scheduler, device)

        val_metrics = evaluate(model, val_loader, criterion, device, threshold=THRESH)
        val_f1 = val_metrics["f1_micro"]

        if val_f1 > best_f1:
            best_f1 = val_f1
            best_state = model.state_dict()

    # Evaluate best model on test set
    final_model = BERTSingleTaskClassifier(n_classes=n_classes).to(device)
    final_model.load_state_dict(best_state)

    test_metrics = evaluate(final_model, test_loader, criterion, device, threshold=THRESH)

    return best_state, best_f1, test_metrics


In [60]:
config = {
    "batch_size": 8,
    "learning_rate": 3e-5,
    "epochs": 5,
    "dropout": 0.3,
    "threshold": 0.5
}

model, best_f1, test_metrics = run_single_experiment(
    config,
    X_trainval, y_trainval,
    X_test, y_test,
    tokenizer,
    n_classes,
    device,
)

Evaluating: 100%|██████████| 51/51 [00:04<00:00, 11.14it/s]


In [62]:
torch.save(model, "best_bert_keywords.pth")
test_metrics

{'loss': 1.11071370863447,
 'f1_micro': 0.18992568125516102,
 'f1_macro': 0.1913345893776925,
 'precision': 0.11219512195121951,
 'recall': 0.6182795698924731,
 'hamming_accuracy': 0.8037214885954382,
 'jaccard_samples': 0.11258177169896347}

## PARAM GRID

In [ ]:
from itertools import product

def param_grid_search(param_grid, X_trainval, y_trainval, X_test, y_test, tokenizer, n_classes, device):

    keys = list(param_grid.keys())
    value_combinations = list(product(*param_grid.values()))

    results = []
    best_f1 = -1
    best_config = None
    best_state = None
    best_test_metrics = None

    for run_id, combo in value_combinations:
        config = dict(zip(keys, combo))
        print("\n=======================================")
        print("Testing config:", config)
        print("=======================================\n")

        state_dict, f1, test_metrics = run_single_experiment(
            config,
            X_trainval, y_trainval,
            X_test, y_test,
            tokenizer,
            n_classes,
            device
        )

        print(test_metrics)

        row = {
            "run_id": run_id,
            **config,
            "val_f1_micro": f1,
            **{f"test_{k}": v for k, v in test_metrics.items()}
        }

        # record results
        results.append(row)

        if f1 > best_f1:
            best_f1 = f1
            best_config = config
            best_state = state_dict
            best_test_metrics = test_metrics


    # Build final best model
    best_model = BERTSingleTaskClassifier(n_classes=n_classes).to(device)
    best_model.load_state_dict(best_state)

    # Print metrics for the winning configuration
    print("\n==================== BEST RESULTS ====================")
    print("Best configuration:", best_config)
    print("\nBest Validation F1-micro:", best_f1)
    
    print("\n===== FINAL TEST METRICS (BEST MODEL) =====")
    for k, v in best_test_metrics.items():
        print(f"{k}: {v:.4f}")
    print("=======================================================\n")

    return best_model, best_config, results, best_test_metrics


In [ ]:
param_grid = {
    "batch_size": [8, 16],
    "learning_rate": [2e-5, 3e-5],
    "epochs": [3],
    "dropout": [0.2, 0.3],
    "threshold": [0.3, 0.5]
}

best_model, best_config, all_results, best_test_metrics = param_grid_search(
    param_grid,
    X_trainval,
    y_trainval,
    X_test,
    y_test,
    tokenizer,
    n_classes,
    device
)

print("\nBest config:", best_config)
print("\nResults:", all_results)


results_df = pd.DataFrame(all_results)
results_df.to_csv("BERT/param_grid/param_grid_results_keywords.csv", index=False)


Testing config: {'batch_size': 8, 'learning_rate': 2e-05, 'epochs': 3, 'dropout': 0.2, 'threshold': 0.3}



Evaluating: 100%|██████████| 51/51 [00:04<00:00, 11.38it/s]


{'loss': 1.1554433095688914, 'f1_micro': 0.08818298890047219, 'f1_macro': 0.08764758153121638, 'precision': 0.04619931889738482, 'recall': 0.9663978494623656, 'hamming_accuracy': 0.2562525010004002, 'jaccard_samples': 0.04665053256582751}

Testing config: {'batch_size': 8, 'learning_rate': 2e-05, 'epochs': 3, 'dropout': 0.2, 'threshold': 0.5}



Evaluating: 100%|██████████| 51/51 [00:04<00:00, 11.61it/s]


{'loss': 1.1465506354967754, 'f1_micro': 0.15152926564198105, 'f1_macro': 0.15657267636834052, 'precision': 0.08565911883447429, 'recall': 0.6559139784946236, 'hamming_accuracy': 0.726640656262505, 'jaccard_samples': 0.08364871773460827}

Testing config: {'batch_size': 8, 'learning_rate': 2e-05, 'epochs': 3, 'dropout': 0.3, 'threshold': 0.3}



...

{'loss': 1.1570028570982127, 'f1_micro': 0.15138238404592838, 'f1_macro': 0.16080658166183057, 'precision': 0.08527659574468086, 'recall': 0.6733870967741935, 'hamming_accuracy': 0.7190376150460185, 'jaccard_samples': 0.08485712443681215}

Testing config: {'batch_size': 16, 'learning_rate': 3e-05, 'epochs': 3, 'dropout': 0.3, 'threshold': 0.3}



Evaluating: 100%|██████████| 26/26 [00:04<00:00,  5.81it/s]


{'loss': 1.1676436227101545, 'f1_micro': 0.08425205988162934, 'f1_macro': 0.08527720449616617, 'precision': 0.04402668283808369, 'recall': 0.9758064516129032, 'hamming_accuracy': 0.21058423369347734, 'jaccard_samples': 0.044337517507536395}

Testing config: {'batch_size': 16, 'learning_rate': 3e-05, 'epochs': 3, 'dropout': 0.3, 'threshold': 0.5}



Evaluating: 100%|██████████| 22/22 [00:03<00:00,  6.06it/s]
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 59e52431-2bc7-4d1b-b546-f332fd7d040a)')' thrown while requesting HEAD https://huggingface.co/distilbert-base-uncased/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Evaluating: 100%|██████████| 26/26 [00:04<00:00,  6.01it/s]


{'loss': 1.1699710511244261, 'f1_micro': 0.14854586129753916, 'f1_macro': 0.15314540783121752, 'precision': 0.08354302969300453, 'recall': 0.6693548387096774, 'hamming_accuracy': 0.7144357743097238, 'jaccard_samples': 0.08348704912560886}

==================== BEST RESULTS ====================
Best configuration: {'batch_size': 8, 'learning_rate': 3e-05, 'epochs': 3, 'dropout': 0.3, 'threshold': 0.5}

Best Validation F1-micro: 0.17172944511948157

===== FINAL TEST METRICS (BEST MODEL) =====
loss: 1.1109
f1_micro: 0.1694
f1_macro: 0.1745
precision: 0.0971
recall: 0.6626
hamming_accuracy: 0.7581
jaccard_samples: 0.0975


Best config: {'batch_size': 8, 'learning_rate': 3e-05, 'epochs': 3, 'dropout': 0.3, 'threshold': 0.5}

Results: [({'batch_size': 8, 'learning_rate': 2e-05, 'epochs': 3, 'dropout': 0.2, 'threshold': 0.3}, 0.08925067228723017), ({'batch_size': 8, 'learning_rate': 2e-05, 'epochs': 3, 'dropout': 0.2, 'threshold': 0.5}, 0.15941765241128297), ({'batch_size': 8, 'learning_rate'

## GENRES PREDICTION

In [15]:
def predict_keywords(text, model, tokenizer, mlb, device, threshold=0.5, top_k=5):
    model.eval()
    
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    
    # Get top-k predictions
    top_indices = np.argsort(probs)[::-1][:top_k]
    top_keywords = [(mlb.classes_[i], probs[i]) for i in top_indices if probs[i] > threshold]
    
    return top_keywords


In [64]:
keywords_map_df = pd.read_csv("../keywords_id_to_name_mapping.csv")
id_to_keyword = dict(zip(keywords_map_df["id"], keywords_map_df["name"]))

In [65]:
def ids_to_names(ids, mapping):
    return [mapping[i] for i in ids if i in mapping]

In [66]:
import random

def show_sample_predictions(
    X_test, 
    y_test, 
    model, 
    tokenizer, 
    mlb, 
    id_to_keyword, 
    device, 
    num_samples=5
):
    indices = random.sample(range(len(X_test)), num_samples)

    for idx in indices:
        sample_text = X_test[idx]

        # Actual keyword IDs - names
        actual_ids = mlb.inverse_transform(y_test[idx:idx+1])[0]
        actual_keywords = [id_to_keyword[i] for i in actual_ids if i in id_to_keyword]

        # Predicted keyword IDs + probabilities
        predicted_raw = predict_keywords(sample_text, model, tokenizer, mlb, device)
        predicted_keywords = [
            (id_to_keyword.get(kid, f"UNKNOWN_{kid}"), prob)
            for kid, prob in predicted_raw
        ]

        print("\n" + "-" * 60)
        print(f"SAMPLE {idx} ")
        print(f"Text:\n{sample_text[:350]}...")
        print("\nActual keywords:")
        print(actual_keywords)

        print("\nPredicted keywords (probabilities):")
        for kw, prob in predicted_keywords:
            print(f"  - {kw}: {prob:.4f}")


In [70]:
model = BERTSingleTaskClassifier(n_classes=n_classes).to(device)
model.load_state_dict(torch.load("best_bert_keywords.pth"))
model.eval()

show_sample_predictions(
    X_test=X_test,
    y_test=y_test,
    model=model,
    tokenizer=tokenizer,
    mlb=mlb,
    id_to_keyword=id_to_keyword,
    device=device,
    num_samples=5
)


------------------------------------------------------------
SAMPLE 69 
Text:
the sequel to house of 1000 corpses the firefly family are ambushed at their isolated home by sheriff wydell and a squad of armed men guns blazing yet only otis and his sister baby manage to escape the barrage of bullets unharmed hiding out in a backwater motel the wanted siblings wait to rendezvous with their errant father captain spaulding killin...

Actual keywords:
['sequel', 'murder', 'serial killer', 'violence']

Predicted keywords (probabilities):
  - escape: 0.7302
  - sequel: 0.7086
  - violence: 0.6994
  - revenge: 0.6670
  - remake: 0.6369

------------------------------------------------------------
SAMPLE 0 
Text:
the story of florence foster jenkins a new york heiress who dreamed of becoming an opera singer despite having a terrible singing voice...

Actual keywords:
['biography']

Predicted keywords (probabilities):
  - musical: 0.8255
  - independent film: 0.6752
  - new york: 0.6459
  - wedd

## 5-FOLD VALIDATION

In [16]:
from sklearn.model_selection import KFold

def cross_validate_bert(
    X, y, config, tokenizer, n_classes, device,
    n_splits=5,
    save_each_model=True
):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold_results = []

    print(f"\n========== STARTING {n_splits}-FOLD CROSS VALIDATION ==========\n")

    for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
        print(f"\n==================== FOLD {fold}/{n_splits} ====================\n")

        X_train = [X[i] for i in train_idx]
        y_train = y[train_idx]

        X_val = [X[i] for i in val_idx]
        y_val = y[val_idx]

        # Run training for this fold
        best_state, val_f1, test_metrics = run_single_experiment(
            config=config,
            X_trainval=X_train,
            y_trainval=y_train,
            X_test=X_val,          # Validation acts as test for this fold
            y_test=y_val,
            tokenizer=tokenizer,
            n_classes=n_classes,
            device=device
        )

        fold_results.append(test_metrics)
        print(f"Fold {fold} F1_micro: {test_metrics['f1_micro']:.4f}")

        if save_each_model:
            torch.save(best_state, f"best_bert_keywords_fold{fold}.pth")
            print(f"✓ Saved best fold-{fold} model → best_fold_{fold}.pth")

    print("\n========== CROSS-VALIDATION COMPLETE ==========")

    # Compute average metrics
    avg = {}
    for k in fold_results[0].keys():
        avg[k] = float(np.mean([m[k] for m in fold_results]))

    print("\n====== AVERAGE CROSS-VALIDATION METRICS ======")
    for k, v in avg.items():
        print(f"{k}: {v:.4f}")

    return fold_results, avg


In [20]:
params = {'batch_size': 8, 'learning_rate': 3e-05, 'epochs': 5, 'dropout': 0.3, 'threshold': 0.5}

cv_results, cv_avg = cross_validate_bert(
    X=texts,
    y=labels_encoded,
    config=params,
    tokenizer=tokenizer,
    n_classes=n_classes,
    device=device,
    n_splits=5,
    save_each_model=True
)


========== STARTING 5-FOLD CROSS VALIDATION ==========


==================== FOLD 1/5 ====================



Evaluating: 100%|██████████| 68/68 [00:06<00:00, 10.97it/s]


Fold 1 F1_micro: 0.1923
✓ Saved best fold-1 model → best_fold_1.pth

==================== FOLD 2/5 ====================



Evaluating: 100%|██████████| 68/68 [00:05<00:00, 11.51it/s]


Fold 2 F1_micro: 0.1872
✓ Saved best fold-2 model → best_fold_2.pth

==================== FOLD 3/5 ====================



...

Fold 3 F1_micro: 0.1790
✓ Saved best fold-3 model → best_fold_3.pth

==================== FOLD 4/5 ====================



Evaluating: 100%|██████████| 68/68 [00:06<00:00, 11.08it/s]


Fold 4 F1_micro: 0.1843
✓ Saved best fold-4 model → best_fold_4.pth

==================== FOLD 5/5 ====================



Evaluating: 100%|██████████| 68/68 [00:05<00:00, 11.72it/s]


Fold 5 F1_micro: 0.1805
✓ Saved best fold-5 model → best_fold_5.pth

========== CROSS-VALIDATION COMPLETE ==========

====== AVERAGE CROSS-VALIDATION METRICS ======
loss: 1.0778
f1_micro: 0.1847
f1_macro: 0.1888
precision: 0.1083
recall: 0.6278
hamming_accuracy: 0.8014
jaccard_samples: 0.1071


In [ ]:
results_path = "results/cross_val_keywords_metrics.csv"
results_df = pd.DataFrame(cv_results)
results_df.to_csv(results_path, index=False)

print(f"Wyniki cross-validation zapisane do: {results_path}")

Wyniki cross-validation zapisane do: results/cross_val_genres_metrics.csv
